In [1]:
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import matplotlib.pyplot as plt

%matplotlib qt5

In [2]:
import configparser, os

# Génération du fichier de configuration

## Lecture des données igoo / config

In [3]:
igloofile = "/home/pleroy/DATA/DIADEM/RobotArm/3D/igloo.txt"
iniFile = "/home/pleroy/DATA/DIADEM/RobotArm/cloudScanSweep.ini"
axes = ["1", "3", "4"] # CACENDRA => 1 azimuth, 2 translation, 3 roll, 4 polar
config = configparser.ConfigParser()
config.optionxform = str # case senstitive
config.read(iniFile)
theta_phi = np.genfromtxt(igloofile)

In [4]:
def buildFile(outputfile, az_pr_rl, config, axes):
    header = "".join([f"{key}={config['header'][key]}\n" for key in config['header']])
    with open(outputfile, 'w') as file:  # Use file to refer to the file object
        file.write("Ant32 CloudScan Setup File V4.1\n")
        file.write(header)
        file.write("#CloudList\n")
        file.write(" ".join(axes) + "\n")
        for row in az_pr_rl:
            file.write(f"{row[0]:.2f} {row[1]:.2f} {row[2]:.2f}\n")

## igloo V

In [5]:
# add a row for the polarization axis, constant value 0° or 90°
polarVal = 0
polar = np.ones((theta_phi.shape[0], 1)) * polarVal

In [6]:
outputfile = os.path.join("/home/pleroy/DATA/DIADEM", "iglooV.css")
az_rl_pl = np.c_[theta_phi, polar]
buildFile(outputfile, az_rl_pl, config, axes)

## roulis 360 V / igloo V / igloo H / roulis 360 H

In [7]:
polarV = np.ones((theta_phi.shape[0], 1)) * 0
polarH = np.ones((theta_phi.shape[0], 1)) * 90
igloo_V = np.c_[theta_phi, polarV]
igloo_H = np.c_[theta_phi[::-1,:], polarH]

In [8]:
rl = np.arange(0, 360, 3.6).reshape(-1, 1)
rl_V = np.c_[np.zeros(rl.shape), rl, np.zeros(rl.shape)]
rl_H = np.c_[np.zeros(rl.shape), rl, np.ones(rl.shape) * 90]

In [9]:
outputfile = os.path.join("/home/pleroy/DATA/DIADEM", "rlV_iglooV_iglooH_rlH.css")
az_rl_pl = np.r_[rl_V, igloo_V, igloo_H, rl_H]
buildFile(outputfile, az_rl_pl, config, axes)

## roulis 360 V / igloo V / igloo H / roulis 360 H

In [42]:
polarV = np.ones((theta_phi.shape[0], 1)) * 0
polarH = np.ones((theta_phi.shape[0], 1)) * 90
igloo_V = np.c_[theta_phi, polarV]
igloo_H = np.c_[theta_phi, polarH]
igloo_VH = None

In [43]:
for theta in sorted(set(theta_phi[:, 0])):
    idx = np.where(theta_phi[:, 0] == theta)[0]
    phis = theta_phi[idx, 1]
    print(f"theta {theta}, phis.shape {phis.size}")
    igloo_VH_tmp = np.zeros((phis.size * 2, igloo_V.shape[1]))
    for k, phi in enumerate(phis):
        igloo_VH_tmp[k, :] = [theta, phi, 0]
        igloo_VH_tmp[k + phis.size, :] = [theta, phi, 90]
    if igloo_VH is None:
        igloo_VH = igloo_VH_tmp
    else:
        igloo_VH = np.r_[igloo_VH, igloo_VH_tmp]

theta 0.0, phis.shape 6
theta 7.5, phis.shape 8
theta 15.0, phis.shape 15
theta 22.5, phis.shape 20
theta 30.0, phis.shape 24
theta 37.5, phis.shape 30
theta 45.0, phis.shape 36
theta 52.5, phis.shape 40
theta 60.0, phis.shape 45
theta 67.5, phis.shape 45
theta 75.0, phis.shape 48
theta 82.5, phis.shape 48
theta 90.0, phis.shape 48
theta 97.5, phis.shape 48
theta 105.0, phis.shape 48
theta 112.5, phis.shape 45
theta 120.0, phis.shape 45
theta 127.5, phis.shape 40
theta 135.0, phis.shape 36
theta 142.5, phis.shape 30
theta 150.0, phis.shape 24
theta 157.5, phis.shape 20
theta 165.0, phis.shape 15
theta 172.5, phis.shape 8
theta 180.0, phis.shape 6


In [44]:
rl = np.arange(0, 360, 3.6).reshape(-1, 1)
rl_V = np.c_[np.zeros(rl.shape), rl, np.zeros(rl.shape)]
rl_H = np.c_[np.zeros(rl.shape), rl, np.ones(rl.shape) * 90]

In [45]:
outputfile = os.path.join("/home/pleroy/DATA/DIADEM", "rlV_rlH_iglooVH_rlV_rlH.css")
az_rl_pl = np.r_[rl_V, rl_H, igloo_VH, rl_V, rl_H]
buildFile(outputfile, az_rl_pl, config, axes)

In [48]:
plt.figure()
plt.plot(az_rl_pl[:, 0], '.-')
plt.plot(az_rl_pl[:, 1], '.-')

# Plot des données

In [ ]:
theta = theta_phi[:, 0]
phi = theta_phi[:, 1]
xs = np.sin(theta) * np.cos(phi)
ys = np.sin(theta) * np.sin(phi)
zs = np.cos(theta)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs, ys, zs)
ax.set_xlabel("theta")
ax.set_ylabel("phi")